# Phase 3 - Conversational Task Agent

## OpenSearch connection settings

### Connection to Index

In [1]:
import pprint as pp
import requests
from opensearchpy import OpenSearch
from opensearchpy import helpers
from searchQueries import *

host = '10.10.255.202'
port = 8200
user = 'user213' # group username
password = '1Fz{K6})[w>C8T*yBhbeTr' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
    #, ca_certs = ca_certs_path
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))

{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'user213': {'settings': {'index': {'creation_date': '1654898808645',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user213',
                                    'refresh_interval': '-1',
                                    'uuid': 'bDuaRdMhTNa_b3vSuAfmXg',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user213': {'mappings': {'dynamic': 'strict',
                          'properties': {'description': {'analyzer': 'standard',
                                                         'similarity': 'BM25',
                        

### Close Index

In [15]:
resp = client.indices.close(index = index_name, timeout="600s")
print(resp)

{'acknowledged': True, 'shards_acknowledged': True, 'indices': {'user213': {'closed': True}}}


### Function Defines and load Index Model

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizerIndex(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = modelIndex(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

# Load model from HuggingFace Hub
tokenizerIndex = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
modelIndex = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

### Adding docs to the index was made on the first phase of this project

## Dialog Manager

### User Intent detection and load Intent Model

In [3]:
import os
import numpy as np
import transformers
import json

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

model_finetuned = '/user/data/public/twiz-intent-model'
with open(os.path.join(model_finetuned + '/all_intents.json'), 'r') as all_intents_json:
    all_intents = json.load(all_intents_json) # contains the written out names of intents. also implicitly

tokenizer_name = 'roberta-base'
tokenizerIntent = AutoTokenizer.from_pretrained(tokenizer_name) # loads a tokenizer

modelIntent = AutoModelForSequenceClassification.from_pretrained(model_finetuned, 
                                                           num_labels=len(all_intents)) # Loads the BERT model weights


### Function Defines to calculate each phrase intent and embeddings of user response

In [4]:
def computeIntent(phrase):
    input_encoding = tokenizerIntent.encode_plus(phrase, return_tensors='pt', add_special_tokens=True, max_length = 512, truncation = True)
    outputs = modelIntent(**input_encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    return all_intents[idx]

def identifyProcessResponse(phrase, numRecipe):
    sourceEmb = ['title']
    embedding = "title_embedding"
    query_emb = encode(phrase)
    responseEmb = embeddingsSearch(sourceEmb, embedding, query_emb, client, index_name)
    return responseEmb["hits"]["hits"][numRecipe]["_id"]

## Image Extraction from JSON

In [5]:
from IPython.display import Image, HTML, display

with open("recipes_data.json", "r") as read_file:
    data = json.load(read_file)

images = []

for x in data:
    recipe_id = x
    principal_images = data[str(x)]['images']
    for z in range (len(principal_images)):
        images.append(principal_images [z]['url'])
    instructions = data[str(x)] ['instructions']
    for u in range (len(instructions)):
        stepImages = instructions [u] ['stepImages']
        for a in range (len(stepImages)):
            images.append(stepImages[a]['url'])

### Open all the images - Run only when necessary

In [6]:
from PIL import Image
import requests

import clip

model, preprocess = clip.load("ViT-B/32")

#This code takes about 10 minutes to run
images_open = []
for x in range(len(images)):
    img_open = Image.open(requests.get(images[x], stream=True).raw)
    images_open.append(preprocess(img_open))

### Calculate image_features - Run only when necessary

In [7]:
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import numpy as np

#This code takes about 4 minutes to run
image_input = torch.tensor(np.stack(images_open))
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
image_features /= image_features.norm(dim=-1, keepdim=True)



### Choose best image

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
from IPython.display import Image as im2

def similarity(feature, photo_features):
    similarities = list((feature @ photo_features.T).squeeze(0))
    return similarities

def bestImage (text):
    with torch.no_grad():
        inputs = tokenizer([text[0:76]],  padding=True, return_tensors="pt")
        inputs = processor(text=[text[0:76]], images=None, return_tensors="pt", padding=True)
    text_encoded =  model.get_text_features(**inputs).detach().numpy()

    similarities = similarity(text_encoded, image_features.cpu().numpy())

    bestphoto_idx = sorted(zip(similarities, range(image_features.shape[0])), key=lambda x: x[0], reverse=True) [0] [1]
    return images[bestphoto_idx]

### Auxiliar methods for image printing

In [10]:
def showRecipeImage(id, recipeText):
    recipe = data[str(id)]
    if(len(recipe['images']) == 0):
        url = bestImage(recipeText)
    else:
        url = recipe['images'][0]['url']
    display(HTML(f"""
      <div class = "stepImage">
       <img src="{url}" class="img-responsive" width="300" heigth ="400">
       <br><br>
    """))

def showStepsImage (id, stepNumber, stepText):
    steps = recipe['instructions']
    if(len(steps[stepNumber]['stepImages']) == 0):
        url = bestImage(stepText)
    else:
        url = steps[stepNumber]['stepImages'][0]['url']
    display(HTML(f"""
      <div class = "stepImage">
       <img src="{url}" class="img-responsive" width="300" heigth ="400">
       <br><br>
    """))


## Game Part: Guess the recipe title

### Function defines to calculate text similarity and remove stop words

In [11]:
from torch import Tensor
import spacy
from spacy import displacy
from pathlib import Path

#Calculate similarity
def cos_sim(a: Tensor, b: Tensor):
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

def removeStopWords(phrase):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(phrase)
    newPhrase = ''
    for token in doc:
        if(not token.is_stop and not token.text == 'recipe'):
            newPhrase +=token.text + " "
    return newPhrase


### Function to calculate similarity between a user's sentence and recipe's title

In [12]:
def guessSimilarity(recipeName, phrase):
    recipeName = removeStopWords(recipeName)
    firstSentence = encode(recipeName)
    phrase = removeStopWords(phrase)
    secondSentence = encode(phrase)
    return cos_sim(firstSentence, secondSentence)


### Auxiliar function to print recipe's steps

In [13]:
def showSteps (stepNumber, stepText):
    display(HTML(f"""
    <div class =  "steps">
       <b>Step Number: </b> {stepNumber}
       <br>
       <b>Step Text:</b> {stepText}
       <br>
    """))

## TaskBot 

In [14]:
from random import randrange
print("Hello, I am a TaskBot and I can help you with cooking tasks. Can I help you? If you do not know what to eat and want to play a game to challenge you apetite respond \033[1mPlay game\033[0m!")
print("Respond \033[1mHelp\033[0m for more information.")
intentResponse = ''
userWins = 0
userLoss = 0
while intentResponse != 'AMAZON.StopIntent':
    phrase = input("USER:")
    phrase = phrase.lower()
    intentResponse = computeIntent(phrase);
    if (intentResponse == 'IdentifyProcessIntent'):
        firstRecipePhrase = phrase
        recipeNumber = identifyProcessResponse(phrase, 0)
        recipe = data[str(recipeNumber)]
        print("I know lots of recipes. Here is a tasteful one. Do you want this one?")
        recipeName = recipe['displayName']
        print ('\033[1m' + recipeName + '\033[0m')
        showRecipeImage(recipeNumber, recipeName)
        phrase = input("USER:")
        intentResponse = computeIntent(phrase);
        if intentResponse ==  'AMAZON.NoIntent':
            recipeNumber = identifyProcessResponse(firstRecipePhrase, 1)
            recipe = data[str(recipeNumber)]
            recipeName = recipe['displayName']
            print("Here is another tasteful one. Do you want this one?")
            print ('\033[1m' + recipeName + '\033[0m')
            showRecipeImage(recipeNumber, recipeName)
            phrase = input("USER:")
            intentResponse = computeIntent(phrase);
            if intentResponse == 'AMAZON.NoIntent':
                print("Try to be more specific on the recipe you want.")
            else:
                print ("Do you want to check first the ingredients of " + recipeName +"?")
                phrase = input("USER:")
                intentResponse = computeIntent(phrase);
                if intentResponse == 'AMAZON.YesIntent':
                    print ("These are the ingredients you need:")
                    ingredients = recipe ['ingredients']
                    for x in range(len(ingredients)):
                        print (" -" + str(recipe['ingredients'][x]['displayText'])) 
                print ("Are you ready to start?")
                phrase = input("USER:")
                intentResponse = computeIntent(phrase);
                if intentResponse == 'AMAZON.YesIntent':
                    print ("Okay let's get started! Here is the first step, if you want to continue just say next or another similar thing" +
                               " back to go to the previous step or you can stop at any moment because you are a really bad cooker")
                    stepCounter = 0; 
                    numberOfSteps = len(recipe['instructions']);
                    showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                    showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                    while intentResponse != 'AMAZON.StopIntent':
                        phrase = input("USER:")
                        intentResponse = computeIntent(phrase);
                        if intentResponse == 'NextStepIntent':
                            if stepCounter == numberOfSteps - 1:
                                intentResponse = 'AMAZON.StopIntent'
                            else:
                                stepCounter += 1
                                showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                                showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                        elif intentResponse == 'PreviousStepIntent':
                            if stepCounter == 0:
                                print ("Easyyyyyyyyy, there is nothing do before this, you are still in the first step.")
                            else:
                                stepCounter -= 1
                                showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                                showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                        elif intentResponse == 'AMAZON.StopIntent':
                            print()
                        else:
                            print ("I am not sure I understand. Use next, back or stop or any similar word.")
                intentResponse = ''
                if stepCounter == numberOfSteps -1:
                    print ("You just finished, I hope it tastes good. Is there anything more I can help with?")
                else :
                    print ("No problem on giving up, tell me another thing you want.")        
        elif intentResponse == 'AMAZON.YesIntent':
            print ("Do you want to check first the ingredients of " + recipeName +"?")
            phrase = input("USER:")
            intentResponse = computeIntent(phrase);
            if intentResponse == 'AMAZON.YesIntent':
                print ("These are the ingredients you need:")
                ingredients = recipe ['ingredients']
                for x in range(len(ingredients)):
                    print (" -" + str(recipe['ingredients'][x]['displayText']))
            print ("Are you ready to start?")
            phrase = input("USER:")
            intentResponse = computeIntent(phrase);
            if intentResponse == 'AMAZON.YesIntent' or intentResponse =='StartStepsIntent' :
                print ("Okay let's get started! Here is the first step, if you want to continue just say next or another similar thing" +
                           " back to go to the previous step or you can stop at any moment because you are a really bad cooker")
                stepCounter = 0; 
                numberOfSteps = len(recipe['instructions']);
                showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                while intentResponse != 'AMAZON.StopIntent':
                    phrase = input("USER:")
                    intentResponse = computeIntent(phrase);
                    if intentResponse == 'NextStepIntent':
                        if stepCounter == numberOfSteps - 1:
                            intentResponse = 'AMAZON.StopIntent'
                        else:
                            stepCounter += 1
                            showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                            showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                    elif intentResponse == 'PreviousStepIntent':
                        if stepCounter == 0:
                            print ("Easyyyyyyyyy, there is nothing do before this, you are still in the first step.")
                        else:
                            stepCounter -= 1
                            showSteps(str(stepCounter +1), str(recipe['instructions'][stepCounter]['stepText']))
                            showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                    elif intentResponse == 'AMAZON.StopIntent':
                        print()
                    else:
                        print ("I am not sure I understand. Use next, back or stop or any similar word.")
            intentResponse = ''
            if stepCounter == numberOfSteps -1:
                print ("You just finished, I hope it tastes good. Is there anything more I can help with?")
            else :
                print ("No problem on giving up, tell me another thing you want.")
    elif intentResponse ==  'OutOfScopeIntent':
        print('Welcome to the recipe game, where I show you an image of a recipe and you have to guess what it is! Are you ready to start?')
        phrase = input("USER:")
        intentResponse = computeIntent(phrase);
        if intentResponse ==  'AMAZON.YesIntent':
            print ("Very well, I will then explain how the game works. I will show you a image of a recipe and you can try to guess what recipe it is.")
            print("The first time you fail, it will be shown the ingredients. If you fail the second time, It will be shown each step until you guess it right!")
            print("You lose the game if at the end of the steps, you did not got a close answer")
            recipeNumber = randrange(994)
            recipe = data[str(recipeNumber)]
            recipeName = recipe['displayName']
            showRecipeImage(recipeNumber, recipeName)
            numberOfSteps = len(recipe['instructions']);
            counter = 0
            oldSimValue = 0
            while counter != 1 + numberOfSteps:
                phrase = input("USER:")
                if(computeIntent(phrase) == 'AMAZON.StopIntent'):
                    intentResponse = ''
                    print("I see you do not want to play with me anymore. What can I do for you then?")
                    break
                simValue = guessSimilarity(recipeName, phrase)
                if(simValue.item() >= 0.70):
                    userWins+=1
                    print("Congratsssssss, you won. You have now "+str(userWins)+" wins and "+str(userLoss)+" losses. What else can I do for you? ")
                    intentResponse = ''
                    break
                else:
                    print("Try again!") 
                    if simValue > oldSimValue:
                        print("You are getting closer!")
                    else:
                        print("Colder... You were closer last try")
                    print("Your score is "+ str(round(simValue.item(), 1)*100) +"%")
                    if counter == 0:
                        print ("These are the ingredients of the recipe:")
                        ingredients = recipe ['ingredients']
                        for x in range(len(ingredients)):
                            print (" -" + str(recipe['ingredients'][x]['displayText'])) 
                        stepCounter = 0;
                        oldSimValue = simValue
                    elif stepCounter == numberOfSteps - 1:
                        userLoss+=1
                        print("You lost! The correct recipe title was: " + recipeName + ". You have now "+str(userWins)+" wins and "+str(userLoss)+" losses."+
                              " Better luck next time! Is there anything else I can do for you?")
                        intentResponse = ''
                        break
                    else:
                        numberOfSteps = len(recipe['instructions']);
                        print ("Step " + str(stepCounter +1) + "- " + str(recipe['instructions'][stepCounter]['stepText']))
                        showStepsImage (recipeNumber, stepCounter, str(recipe['instructions'][stepCounter]['stepText']))
                        stepCounter += 1
                        oldSimValue = simValue
                    counter += 1
        else:
            intentResponse = ''
            print("I see that you do not want to play. What can I do for you?")
    elif intentResponse ==  'AMAZON.YesIntent':
        print ("What would you like to eat?")
    elif intentResponse == 'AMAZON.StopIntent':
        print () 
    elif intentResponse == 'AMAZON.NoIntent':
        print ("What can I do for you then? If you want to play game respond Play Game.")
    elif intentResponse == 'AMAZON.HelpIntent':
        print("Here is some suggestions that you can ask me:")
        print(" \033[1m- I want a chicken recipe\033[0m and I will describe all the details of the desired recipe.")
        print(" \033[1m- Play Game\033[0m so we can play a fun game!")
    else :
        print ("Can you be more specific please. Ask me for a recipe or type No if you want to play a game")
    
    
    
print ("I hope I helped you, have a good day")

Hello, I am a TaskBot and I can help you with cooking tasks. Can I help you? If you do not know what to eat and want to play a game to challenge you apetite respond Play game!
Respond Help for more information.


USER: help


Here is some suggestions that you can ask me:
 - I want a chicken recipe and I will describe all the details of the desired recipe.
 - Play Game so we can play a fun game!


USER: I want some chicken


I know lots of recipes. Here is a tasteful one. Do you want this one?
My Favorite Chicken Salad


...

Try again!
Colder... You were closer last try
Your score is 0.0%
You lost! The correct recipe title was: English Roast Chicken. You have now 1 wins and 1 losses. Better luck next time! Is there anything else I can do for you?


USER: no


What can I do for you then? If you want to play game respond Play Game.


USER: goodbyew



I hope I helped you, have a good day
